# Extracting data from the original CKW dataset

### extracting unique ids 

In [1]:
import polars as pl
import os

dataframes = {}
dataframes['ckw01']=pl.read_csv(r'DATA/ckw_opendata_smartmeter_dataset_a_202101.csv.gz')

ids = dataframes['ckw01']['id'].unique()
print(f"Number of Smart Meter IDs are {len(ids)}")
ids_df = pl.DataFrame({"id": ids})

# Save the DataFrame to a CSV file change the location as required
ids_df.write_csv("unique_ids.csv")
#unique_ids.csv now contains the unique smart meter ids in the CKW file used.
#save the unique ids from one file only. from then on use the csv file to extract other months of the same ids


## saving individual files per smart meter id

In [ ]:
dataframes['ckw02']=pl.read_csv(r'DATA/ckw_opendata_smartmeter_dataset_a_202102.csv.gz') #monthly CKW file path
#load the monthly files individually to avoid crashing the system due to memory issues

In [ ]:
ids = pl.read_csv("unique_ids.csv")

In [ ]:
# Dictionary to store the result DataFrames
combined_house_dataframes = {}
i = 0

save_dir = r'DATA/try' #write the location where the files will be saved
# Ensure the save directory exists
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Debug: Print all keys in dataframes
print("Available months in dataframes:", list(dataframes.keys()))

# Process each common ID
for house_id in ids[i:]:  # Use .values to get the actual array of IDs
   
    # Initialize an empty list to store dataframes for the current house ID
    house_data_list = []
    print(f'Processing house ID: {house_id}, {i}')
    
    # Iterate over specific monthly dataframes
    for key, df in list(dataframes.items()):  # Ensure the slice includes existing keys
       
        #house_data = df[df['id'] == house_id].copy()
        house_data = df.filter(pl.col('id')==house_id)

        # Check for duplicate rows within each day's data
        house_data = house_data.unique(subset=['timestamp', 'id'])
        #dropping id column to reduce size
        house_data = house_data.drop('id')
        
        # Append the filtered dataframe to the list
        house_data_list.append(house_data)
    
    if house_data_list:
        # Concatenate all dataframes in the list
        concatenated_house_data = pl.concat(house_data_list)
    
        # Store the concatenated dataframe in the dictionary
        combined_house_dataframes[house_id] = concatenated_house_data
    
        file_name = f'{i}_data.csv' #change name as required
        file_path = os.path.join(save_dir, file_name)
        df1 = concatenated_house_data.sort(by='timestamp')
        df1.write_csv(file_path)
        #print(f"Data for house ID: {house_id} saved to {file_name}")

        # Verify the number of rows for each house ID
        #print(f"House ID {house_id} has {len(df1)} rows. Expected 8832 rows.")
        
    else:
        print(f"No data found for house ID: {house_id} in the specified dataframes.")
        
    i += 1
print("DONE")
#the individual files for the months loaded is saved.

## Adding subsequent months data to existing file
The process is done in batching due to memory constraints

In [8]:
# Assuming combined_house_dataframes already contains the data for the first 4 months for each house_id
save_dir= r'C:\DATA\ckw\spring'

# Process each common ID
for house_id in combined_house_dataframes:
    # Get the existing concatenated dataframe for the current house_id
    concatenated_house_data = combined_house_dataframes[house_id]
    
    # Initialize an empty list to store dataframes for the next 4 months
    next_house_data_list = []
    print(f'Adding next 4 months for house ID: {house_id}')
    i=0
    # Iterate over the next 4 monthly dataframes
    for key, df in list(dataframes.items())[3:5]:  # Adjust the slice [4:8] for the next 4 months
        print(f'Processing dataframe: {key}')
        # Filter the current dataframe for the current house ID
        house_data = df[df['id'] == house_id].copy()

        # Check for duplicate rows within each day's data
        house_data = house_data.drop_duplicates(subset=['timestamp', 'id'])
        
        # Append the filtered dataframe to the list
        next_house_data_list.append(house_data)
    
    # Concatenate all dataframes for the next 4 months
    concatenated_next_data = pd.concat(next_house_data_list, ignore_index=True)
    
    # Concatenate with the existing data for the house_id
    updated_house_data = pd.concat([concatenated_house_data, concatenated_next_data], ignore_index=True)
    
    # Update the dictionary entry for the house_id
    combined_house_dataframes[house_id] = updated_house_data

# Now combined_house_dataframes contains the data for the first 8 months for each house_id


In [29]:
# Save each concatenated DataFrame to a CSV file
i=0
for house_id, df in combined_house_dataframes.items():
    file_name = f'{i}_data.csv'
    df1=df.sort_values(by='timestamp')
    df1.to_csv(file_name, index=False)
    print(f"Data for house ID: {house_id} saved to {file_name}")

    # Verify the number of rows for each house ID
    print(f"House ID {house_id} has {len(df1)} rows. Expected 35040 rows.") $
    #for the whole year 365 day x 24 hours x 4 in each hour
    i=i+1

## filling the missing data with 0 if required

In [ ]:
#only for winter
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

winter_timestamp_range = pl.datetime_range(datetime(2021,12,1), datetime(2022,2,28,23,45), "15m",eager=True)
# Function to ensure the dataset has the full range of timestamps
def fill_missing_timestamps(df, full_range):
    # Convert the timestamp column to datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    
    # Create a DataFrame with the full range of timestamps
    full_range_df = pd.DataFrame({'timestamp': full_range})
    full_range_df = pd.to_datetime(full_range_df['timestamp'], utc=True)
    # Merge the original DataFrame with the full range DataFrame
    merged_df = pd.merge(full_range_df, df, on='timestamp', how='left')
    
    # Fill missing values with 0
    merged_df['value_kwh'].fillna(0, inplace=True)
    
    return merged_df
if season == "winter":
    for i,file_path in enumerate(file_paths):
        df = pd.read_csv(file_path)
        #df = df.tail( df.shape[0] -3)
        #df.write_csv(file_path)
        #print(f"no. of rows in {i} =",len(df))
        if len(df)<8640:
            # Ensure the dataset has the full range of timestamps and fill missing values
            df_filled = fill_missing_timestamps(df, winter_timestamp_range)
            # Save the updated dataset back to CSV
            df_filled.to_csv(file_path, index=False)